In [1]:
import numpy as np
import pandas as pd
import pickle as p
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os

In [2]:
dir = "../results4"

In [3]:
files = os.listdir(dir)

In [4]:
architecture = np.array([128, 128, 128, 10])

In [5]:
def avg_usage(rates, architecture):
    weights = architecture / np.sum(architecture)
    avg = np.dot(rates, weights)
    return avg

In [6]:
avg_fn = lambda x: avg_usage(x, architecture)

In [7]:
results = [p.load(open(os.path.join(dir, f), "rb")) for f in files]

In [8]:
"random_removal" in results[0].keys()

True

In [9]:
"random_removal" in results[1].keys()

True

In [10]:
def reduce_result(result):
    reduced = {}
    normal_keys = ["mask angle", "cross inhibit", "random_removal", "accuracy"]

    for k in normal_keys:
        if k in result.keys():
            reduced[k] = float(result[k])
        else:
            reduced[k] = 0.0

    if reduced["mask angle"] > 0.0:
        reduced["reduction"] = "mask angle"
    elif reduced["cross inhibit"] > 0.0:
        reduced["reduction"] = "cross inhibit"
    elif reduced["random_removal"] > 0.0:
        reduced["reduction"] = "random"
    else:
        reduced["reduction"] = "none"
    
    avg_keys = ["matrix usage", "firing rates"]
    for k in avg_keys:
        reduced[k] = avg_fn(result[k])

    reduced["accuracy spiking"] = float(np.max(result["accuracy_spiking"]))
    
    return reduced

In [11]:
reduce_result(results[0])

{'mask angle': 0.05,
 'cross inhibit': 0.0,
 'random_removal': 0.0,
 'accuracy': 0.8429999947547913,
 'reduction': 'mask angle',
 'matrix usage': 0.8471650469121594,
 'firing rates': 0.7367091117554456,
 'accuracy spiking': 0.8233000040054321}

In [12]:
reduced_results = list(map(reduce_result, results))

In [13]:
df = pd.DataFrame(reduced_results)

In [14]:
df

,mask angle,cross inhibit,random_removal,accuracy,reduction,matrix usage,firing rates,accuracy spiking
0,0.050,0.0,0.0,0.8430,mask angle,0.847165,0.736709,0.8233
1,0.200,0.0,0.0,0.8277,mask angle,0.488492,0.307378,0.1248
2,0.150,0.0,0.0,0.8398,mask angle,0.596774,0.456275,0.5258
3,0.100,0.0,0.0,0.8435,mask angle,0.719204,0.583297,0.7689
4,0.125,0.0,0.0,0.8421,mask angle,0.656975,0.511075,0.6953
5,0.025,0.0,0.0,0.8432,mask angle,0.914861,0.804482,0.8306
6,0.075,0.0,0.0,0.8440,mask angle,0.782490,0.675844,0.8026


In [15]:
ma_z = df["mask angle"] == 0
ci_z = df["cross inhibit"] == 0
rnd_z = df["random_removal"] == 0

In [16]:
ma_i = ci_z * rnd_z
ci_i = ma_z * rnd_z
rnd_i = ma_z * ci_z
ctrl_i = ci_z * rnd_z * ma_z

In [17]:
ma = df[ma_i].sort_values("accuracy spiking")
ci = df[ci_i].sort_values("accuracy spiking")
rnd = df[rnd_i].sort_values("accuracy spiking")

In [18]:
baseline = 0.8432

In [19]:
baseline

0.8432

In [20]:
spk_a = 0.8432

In [21]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x= 1 - ma["firing rates"], y=ma["accuracy spiking"] / spk_a, name="Explicit"))
fig3.add_trace(go.Scatter(x= 1 - ci["firing rates"], y=ci["accuracy spiking"] / spk_a, name="Inhibitory"))
fig3.add_trace(go.Scatter(x= 1 - rnd["firing rates"], y=rnd["accuracy spiking"] / spk_a, name="Random"))

fig3.update_layout(title="Relative Accuracy by Sparsity",
                  xaxis_title="Sparsity",
                  yaxis_title="Relative Accuracy, Spiking",
                  yaxis_range = [0,1.1],
                  xaxis_range = [-0.05, 1.05],
                  width = 900,
                  height = 600,
                  font = dict({"size": 16,}))

In [24]:
px.scatter(ma, x = "mask angle", y = "accuracy spiking")

In [14]:
px.scatter(df, x="firing rates", y="accuracy spiking", color="reduction")